In [1]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from pprint import pprint

In [2]:
# загрузка датасета и отделение слов только менеджера(за ненадобностью проверки слов клиента в задании)

data = pd.read_csv('../test_data.csv')
data = data[data['role'] == 'manager']
data.head()

,dlg_id,line_n,role,text
1,0,1,manager,Алло здравствуйте
3,0,3,manager,Меня зовут ангелина компания диджитал бизнес з...
5,0,5,manager,Угу ну возможно вы рассмотрите и другие вариан...
8,0,8,manager,Угу а на что вы обращаете внимание при выборе
11,0,11,manager,Что для вас приоритет


In [3]:
import spacy
from spacy.displacy import serve as ss
nlp = spacy.load("ru_core_news_sm")
import re
import nltk
import pymorphy2

morf = pymorphy2.MorphAnalyzer()

# класс объектов для каждой строки

class NLP:
    
    def __init__(self, origin):  
        self.origin = origin                         # оригинальный текст из данных
        self.lemma = self.func(self.origin)          # лемматизированный текст строки
        self.name_maneger = self.name_func(self.lemma) # имя менеджера
        self.greeting = self.greet(self.origin)      # приветствие
        self.farewell = self.farel(self.origin)      # прощание
        self.company = self.compan(self.lemma)       # название компании
        
    # функция лемматизации    
    def func(self, text):
        doc = nlp(text)
        return ' '.join([i.lemma_ for i in doc])
    
    # поиск имени менеджера
    def name_func(self, text):
        if re.search(r'меня \w+ звать', text):
            return re.findall(r'меня \w+ звать', text)[0].split()[-2]
        elif re.search(r'меня звать \w', text):
            return re.findall(r'меня звать \w+', text)[0].split()[-1]
        elif re.search(r'это \w+', text):
            name = re.findall(r'это \w+', text)[0].split()[-1]
            if 'Name' in morf.parse(name)[0].tag:
                return name
            else:
                return None
        else:
            return None
    
    # поиск приветствия
    def greet(self, text):
        if re.search(r'здравств|добрый день|добрый вечер|доброе утро', text):
            return re.findall(r'здравств.*|добрый день|добрый вечер|доброе утро', text)[0]
        else:
            return False
    
    # поиск прощания
    def farel(self, text):
        if re.search(r'до свидан|всего хорош|всего добр', text):
            return re.findall(r'до свидан.*|всего хорош.*|всего добр.*', text)[0]
        else:
            return False   
    
    # поиск названия компании
    def compan(self, text):
        
        if re.search(r'компания', text):
            name = re.findall(r'компания \w+ \w+', text)[0].split()
            if 'NOUN' in morf.parse(name[1])[0].tag:
                return name[1]
            elif 'NOUN' in morf.parse(name[2])[0].tag:
                return ' '.join(name[1:])
            else:
                return None
        else:
            return None

2022-08-30 20:01:45.648780: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-30 20:01:45.648799: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2022-08-30 20:01:46.635063: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-30 20:01:46.635314: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-08-30 20:01:46.635368: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.s

In [4]:
# сбор информации по данным
dict_s = {}                         # словарь извлеченных данных 

# извлечение номеров диалогов из столбца 'dlg_id' и дальнейшая обработка по каждому номеру диалога
for i in data['dlg_id'].unique():
    data_iter = data[data['dlg_id'] == i]      # данные диалога одного диалога
    
    # установка в словарь значений по умолчанию со всеми отрицательными критериями
    dict_s[f'dlg_id_{i}'] = {'name_maneger': 'Не представился', 
                             'greeting': False, 'farewell': False,
                             'criterion':False, 'company': 'Не названа'}  
    # поиск необходимых слов в каждой строке диалога
    for string in data_iter['text']:
        # создание класса строки 
        out = NLP(string)  
        
        # изменение данных в итоговом словаре если в параметрах класса не отрицательные(отсутствующие) значения
        if out.name_maneger != None:
            dict_s[f'dlg_id_{i}']['name_maneger'] = out.name_maneger
            
        if out.greeting != False:
            dict_s[f'dlg_id_{i}']['greeting'] = out.greeting
        if out.farewell != False:
            dict_s[f'dlg_id_{i}']['farewell'] = out.farewell
        if out.company != None:
            dict_s[f'dlg_id_{i}']['company'] = out.company
        
        # проверка выполнения требуемого критерия по приветствию и прощанию менеджера
        if dict_s[f'dlg_id_{i}']['greeting'] != False and dict_s[f'dlg_id_{i}']['farewell'] != False:
            dict_s[f'dlg_id_{i}']['criterion'] = True
        

In [5]:
# результат
pprint(dict_s)

{'dlg_id_0': {'company': 'диджитал бизнес',
              'criterion': True,
              'farewell': 'до свидания',
              'greeting': 'здравствуйте',
              'name_maneger': 'ангелина'},
 'dlg_id_1': {'company': 'диджитал бизнес',
              'criterion': True,
              'farewell': 'всего хорошего',
              'greeting': 'здравствуйте',
              'name_maneger': 'ангелина'},
 'dlg_id_2': {'company': 'диджитал бизнес',
              'criterion': False,
              'farewell': False,
              'greeting': 'здравствуйте',
              'name_maneger': 'ангелина'},
 'dlg_id_3': {'company': 'китобизнес',
              'criterion': True,
              'farewell': 'всего доброго',
              'greeting': 'добрый день',
              'name_maneger': 'максим'},
 'dlg_id_4': {'company': 'Не названа',
              'criterion': False,
              'farewell': 'до свидания',
              'greeting': False,
              'name_maneger': 'Не представился'},
 

In [6]:
# визуальная проверка извлеченных данных

number_dlg = 0

for i in data[data['dlg_id'] == number_dlg]['text']:
    print(i)


Алло здравствуйте
Меня зовут ангелина компания диджитал бизнес звоним вам по поводу продления лицензии а мы с серым у вас скоро срок заканчивается
Угу ну возможно вы рассмотрите и другие варианты видите это хорошая практика сравнивать
Угу а на что вы обращаете внимание при выборе
Что для вас приоритет
Ну у вас срок заканчивается поэтому мы набрали + ко всему
А так нет не только поэтому просто я обратила внимание и вы пользуетесь виджетами мы там с вами сотрудничали
А если вы 19 являетесь то лучше то идти бесплатно
Ага хорошо
Индивидуальным поэтому не все то есть сотрудничают с кем то сейчас да кому то например не с кем сейчас справляться кто то сравнивал
Да мы услышали вас спасибо за рекомендации
Вот понимаю да но дело в том что в основном да за месяц
Необходимо обзванивать
Хорошо я данную информацию тогда зафиксирую передам будем иметь ввиду
И вам спасибо большое за обратную связь
Всего хорошего до свидания


In [7]:
# создание итоговой таблицы данных
# 
data_out = data.copy()

# добавление пустых столбцов в датафрейм
for i in list(dict_s['dlg_id_0'].keys()):
    data_out[i] = 0

# заполнение полей из словаря значений
for dlg_id in data['dlg_id'].unique():
    for column in list(dict_s['dlg_id_0'].keys()):
        data_out[column][data_out['dlg_id'] == dlg_id] = dict_s[f'dlg_id_{dlg_id}'][column]
    

In [8]:
# сохранение в файл
name_file = 'data_parse.csv'
data_out.to_csv(name_file, index=False)